# Bus Schedule Optimization based on demand at Metro Stations

        Analyzing the available ticketing data, the passenger travel distribution is calculated for each bustop.This include how many people move from one bustop to another during a given time period.
    
        The demand at the bustops near the metros are integrated in the time slots 5-6 AM,6-7 am,8-10 am,10-12 am,12 noon - 2pm,2-4pm,4-7pm,7-10pm. This can roughly mock the destinations of passengers exiting at the Metro station.The requirement at each metro station is calculated by dividing the exit data of metro station according to the destination distribution raio obtained as above. This is done for data integrated over 1mins

## Obtaining Existing Time Table from extracted data

In [4]:
f=open('opti_data/boarding_time_based_tschedule_fnames','r')
l=f.readline()
fnames=[]
while l:
    l=l[:-1]
    fnames.append(l)
    l=f.readline()
f.close()

rt_stage_weedayTT=dict()
cnt=0
L=len(fnames)
# for rout in [fnames[0]] :
for rout in fnames :
    cnt+=1
    if cnt%25==0:
        print(cnt,"/",L)
#     if t>2:
#         break
    fr=open('opti_data/boarding_time_based_tschedule/'+rout,'r')
    rt_stage_weedayTT.update({rout:dict()})
    li=fr.readline()
    while li:
        it=li[:-1].split(",")
        wday=it.pop(0)
        stage=it.pop(0)
        if stage not in rt_stage_weedayTT[rout].keys():
            rt_stage_weedayTT[rout].update({stage:dict()})
        if wday not in rt_stage_weedayTT[rout][stage].keys():
            rt_stage_weedayTT[rout][stage].update({wday:[]})
        for time_str in it:
            hr,mi=time_str.split(":")
            rt_stage_weedayTT[rout][stage][wday].append((int(hr)*60+int(mi))*60)
        li=fr.readline()    

25 / 595
50 / 595
75 / 595
100 / 595
125 / 595
150 / 595
175 / 595
200 / 595
225 / 595
250 / 595
275 / 595
300 / 595
325 / 595
350 / 595
375 / 595
400 / 595
425 / 595
450 / 595
475 / 595
500 / 595
525 / 595
550 / 595
575 / 595


## Reading out the Routes and Stages in them

In [6]:
crstg=open("opti_data/route_stage_map",'r') # route stage map
rout_stage_map=dict()
l=crstg.readline()
while l:
    l=l.replace("\n","")
    it=l.split(",")
    rout_stage_map.update({it[0]:it[1:]})
    l=crstg.readline()
crstg.close()

f=open("opti_data/stage_route_map",'r') # stage rout map
stage_rout_map=dict()
l=f.readline()
while l:
    l=l.replace("\n","")
    it=l.split(",")
    stage_rout_map.update({it[0]:it[1:]})
    l=f.readline()
f.close()

## Defining the penality function for inferance

In [7]:
def penality_weight(dt,pax_cnt,total_tt=0): 
                            # percent of travel time 
                            # distace petrol pollution and frequency
    return dt*pax_cnt/60

## Getting the Penality from the Demands at Metros

    The demands from metro are save in the folder 'opti_data/mrtro_simul_files/' [ obtained fron the methord described earlier]

In [8]:
def get_penality_from_stage2(stage,passing_routes,rt_stage_weekdaytt,rt_stg_map):
    t_width=300
    f=open('opti_data/mrtro_simul_files/'+stage,'r')
#     f=open('opti_data/stage_to_stage_pax60s/'+stage,'r')
    l=f.readline()
    destn_pax_count=dict()
    while l:
        it=l.split(",")
        dest=it.pop(0)
#         print(it)
        destn_pax_count.update({dest:[float(i) for i in it]})     
#         print(dest,len(destn_pax_count[dest]))
        l=f.readline()
    f.close()
    no_bus=dict()
    penality=dict()
    for dest in destn_pax_count.keys():
#         print("\t\t DOING ",dest)
        for i in range(len(destn_pax_count[dest])):
            if destn_pax_count[dest][i]==0:
                continue
            t_pax=i*t_width
            tmin=10e7
            tlast_min=0
            
            min_rt=""
#             print("non_zero at ",t_pax,len(passing_routes))
            for through_stage_route in passing_routes:
                if dest in rt_stg_map[through_stage_route]:
                    l_last=0
                    dt=0
#                     print(dest,"yep found in ",through_stage_route)
                    for wday in rt_stage_weekdaytt[through_stage_route][stage].keys():
                        for t in rt_stage_weekdaytt[through_stage_route][stage][wday]:
#                             print(t,t_pax,len(rt_stage_weekdaytt[through_stage_route][stage][wday]))
                            l_last=dt
                            dt=t-t_pax
                            if dt>=0:
                                dt=t-t_pax
                                if dt<tmin:
#                                     print("got min rt ",through_stage_route)
                                    min_rt=through_stage_route
                                    tmin=dt
                                    tpen=t_pax
                                    tlast_min=l_last
                                break
            if tmin==10e7:
                if dest not in no_bus.keys():
#                     print("updating no bus at ",t_pax)
                    no_bus.update({dest:t_pax})
                break
            if min_rt not in penality.keys():
#                 print("updating penaliity at ",t_pax)
                penality.update({min_rt:[]})
            penality[min_rt].append((stage,tpen,tlast_min,tmin,destn_pax_count[dest][i]))
    return no_bus,penality

# TWEEKING THE TIME TABLE

     The existing timetable is tweaked accordingly to get an optimized Time table.
     
     The waiting time for passengers getting on each route are saved keeping note of the place and time. This data is used to identify the bus due to which the penality belong.

In [9]:
#TT TWEAKER
from numpy import average
def tweak_tt(rout_stage_wday_t,penality,rt_stage_map):
    L=len(penality)
    cntu=0
    new_rout_stage_wday_tt=dict()
    for rt in rout_stage_wday_t.keys():
        if cntu%32==0:
            print("DOING ",cntu,"/",L)
        if rt not in penality.keys():
            new_rout_stage_wday_tt.update({rt:rout_stage_wday_t[rt]})
            continue
        
        cntu+=1
        
        stg_stg_tavel=dict()
        f=open('opti_data/stage_stage_tavel_time/'+rt,'r')
        l=f.readline()
        while l:    
            it=l.split(",")
            if it[1] not in stg_stg_tavel.keys():
                stg_stg_tavel.update({it[1]:dict()})
            if it[2] not in stg_stg_tavel[it[1]].keys():
                stg_stg_tavel[it[1]].update({it[2]:dict()})
            if it[0] not in stg_stg_tavel[it[1]][it[2]].keys():
                stg_stg_tavel[it[1]][it[2]].update({it[0]:average([float(i) for i in it[3:] ])})
            l=f.readline()
        f.close()
        wt_max=0
        wt_max_id=0
        k=-1
        tme,wt_time,pax_cnt,lost_bs_time,pen=[],[],[],[],[]
        pen_stage=[]
        for i in penality[rt]['wtime'].keys():
            k+=1
            wt=penality[rt]['wtime'][i]
            if wt_max<wt:
                wt_max=wt
                wt_max_id=k
            pax=penality[rt]['pax_cnt'][i]
            tme.append(int(i))
            wt_time.append(int(wt))
            pax_cnt.append(int(pax))
            lost_bs_time.append(int(penality[rt]['lost_time'][i]))
            pen_stage.append((penality[rt]['pen_stage'][i]))
            pen.append(int(penality_weight(wt,pax)))
        
        t_change=wt_time[k]
        t_pivot=tme[k]
        stg_id=rt_stage_map[rt].index(pen_stage[k])

        if abs(lost_bs_time[k])<abs(wt_time[k]):
            t_change=lost_bs_time[k]
            t_pivot=tme[k-1]
            
        dt_minus=0
        dt_plus=0
        for i in range(len(rt_stage_map[rt])-1):
            if i<stg_id:
                try :
                    x=[]
                    
                    for wd in stg_stg_tavel[rt_stage_map[rt][i]][rt_stage_map[rt][i+1]].keys():
                        x.append(average(stg_stg_tavel[rt_stage_map[rt][i]][rt_stage_map[rt][i+1]][wd]))
                    dt_minus+=average(x)
#                     print("dm")
                except:
                    dt_minus+=200
#                     print ("except - ,",end="")
            else:
                try:
                    if rt_stage_map[rt][i+1] not in stg_stg_tavel[rt_stage_map[rt][i]].keys():
                        continue
#                     print("stg_stg_tavel.keys()",stg_stg_tavel.keys())
#                     print("stg_stg_tavel[rt][rt_stage_map[rt][i]].keys",stg_stg_tavel[rt_stage_map[rt][i]].keys())
                    x=[]
#                     print(rt_stage_map[rt][i+1])

                    for wd in stg_stg_tavel[rt_stage_map[rt][i]][rt_stage_map[rt][i+1]].keys():
                        x.append(average(stg_stg_tavel[rt_stage_map[rt][i]][rt_stage_map[rt][i+1]][wd]))
                    dt_plus+=average(x)  
#                     print("dp")
                except:
                    dt_plus+=280
#                     print ("except + ,",end="")
            
        t_change_start=t_pivot-dt_minus
        t_change_end=t_pivot+dt_plus
        new_rout_stage_wday_tt.update({rt:dict()})
        for s in rout_stage_wday_t[rt].keys():
            new_rout_stage_wday_tt[rt].update({s:dict()})
            for w in rout_stage_wday_t[rt][s]:
                new_rout_stage_wday_tt[rt][s].update({w:[]})
                for i in range(len(rout_stage_wday_t[rt][s][w])):
                    if rout_stage_wday_t[rt][s][w][i]>=t_change_start :
                        if rout_stage_wday_t[rt][s][w][i]<=t_change_end:
                            new_t=rout_stage_wday_t[rt][s][w][i]-t_change
                            new_rout_stage_wday_tt[rt][s][w].append(new_t)
                    else:
                        new_rout_stage_wday_tt[rt][s][w].append(rout_stage_wday_t[rt][s][w][i])
    return new_rout_stage_wday_tt

## COMPARING THE PENALTIES

#### MAKNG THE OLD PENALITY FILES

In [10]:
f=open('opti_data/mrtro_simul_files_fnames','r')
l=f.readline()
fnames=[]
while l:
    fnames.append(l[:-1])
    l=f.readline()
f.close()

no_buses=dict()
penalty=dict()
L=len(fnames)
cnt=0
for nme in fnames:
    cnt+=1
    if cnt%25==0:
        print(cnt ,"/" , L , " : ",nme)
    n,p=get_penality_from_stage2(nme,stage_rout_map[nme],rt_stage_weedayTT,rout_stage_map)
    no_buses.update({nme:n})
    penalty.update({nme:p})    
    
rt_penality=dict()
for nme in penalty.keys():
    for rts in penalty[nme].keys():
        if rts not in rt_penality.keys():
            rt_penality.update({rts:[]})
for nme in penalty.keys():
    for rts in penalty[nme].keys():
        for item in penalty[nme][rts]:
            rt_penality[rts].append(item)

            
for rts in rt_penality.keys():
    f=open('opti_data/penality/'+rts,'w')
    for i in rt_penality[rts]:
        f.write(str(i)+"\n")
    f.close()

25 / 135  :  Dwarka Flyover
50 / 135  :  Kapashera Border
75 / 135  :  New Dwarka Road
100 / 135  :  Rail Bhawan Metro Station
125 / 135  :  St. Stephen Hospital


#### MAKING THE NEW PENALITY FILES

In [ ]:
new_tt=tweak_tt2(rt_stage_weedayTT,penality_compiled_old,rout_stage_map)

#### MAKING THE NEW PENALITY FILES

In [ ]:
f=open('opti_data/mrtro_simul_files_fnames','r')
l=f.readline()
fnames=[]
while l:
    fnames.append(l[:-1])
    l=f.readline()
f.close()

no_buses=dict()
penalty=dict()
L=len(fnames)
cnt=0

for nme in fnames:
    cnt+=1
    if cnt%25==0:
        print(cnt ,"/" , L , " : ",nme)
    n,p=get_penality_from_stage2(nme,stage_rout_map[nme],new_tt,rout_stage_map)
    no_buses.update({nme:n})
    penalty.update({nme:p})    
    
rt_penality=dict()
for nme in penalty.keys():
    for rts in penalty[nme].keys():
        if rts not in rt_penality.keys():
            rt_penality.update({rts:[]})
for nme in penalty.keys():
    for rts in penalty[nme].keys():
        for item in penalty[nme][rts]:
            rt_penality[rts].append(item)

            
for rts in rt_penality.keys():
    f=open('opti_data/penality2/'+rts,'w')
    for i in rt_penality[rts]:
        f.write(str(i)+"\n")
    f.close()

#### READING OUT NEW PENALITY FILE INTO dict()  penality_compiled_new

In [ ]:
f=open('opti_data/penality2_fnames','r')
l=f.readline()
fnames=[]
while l:
    fnames.append(l[:-1])
    l=f.readline()
f.close()

penality_compiled_new=dict()

for nme in fnames:
    f=open('opti_data/penality2/'+nme,'r')
    l=f.readline()
    t_wtime=dict()
    t_pax=dict()
    t_ltime=dict()
    pen_stage=dict()
    while l:
        l=l.replace("('","")
        l=l.replace(")\n","")
        l=l.replace("'","")
        it=l.split(",")
        t_pax.update({int(it[1]):int(it[4])})
        if it[2]=='0':
            l=f.readline()
            continue
        t_wtime.update({int(it[1]):int(it[3])})
        t_ltime.update({int(it[1]):int(it[2])})
        pen_stage.update({int(it[1]):it[0]})
        l=f.readline()
    penality_compiled_new.update({nme:dict()})
    penality_compiled_new[nme].update({'wtime':t_wtime})
    penality_compiled_new[nme].update({'pax_cnt':t_pax})
    penality_compiled_new[nme].update({'lost_time':t_ltime})
    penality_compiled_new[nme].update({'pen_stage':pen_stage})

#### MAKING THE PLOTS

In [ ]:
f=open('opti_data/penality2_fnames','r')
l=f.readline()
fnames=[]
while l:
    fnames.append(l[:-1])
    l=f.readline()
f.close()
noy=[]
si=3
print(len(fnames))
print(len(penality_compiled_new))
for k in range(len(fnames)):
#     if k>0:
#         break
    try:
        print(k,"/",len(fnames)," : ",fnames[k])
        x,y,z=[],[],[]
        pen,llost=[],[]
        f,((ax1,ax2,ax3,ax4),(ax5,ax6,ax7,ax8))=plt.subplots(nrows=2,ncols=4)    
        f.set_figheight(8)
        f.set_figwidth(16)
        f.suptitle(fnames[k]+"\n")

        for i in penality_compiled_old[fnames[k]]['wtime'].keys():
            wt=penality_compiled_old[fnames[k]]['wtime'][i]
            pax=penality_compiled_old[fnames[k]]['pax_cnt'][i]
            x.append(int(i/60))
            y.append(int(wt/60))
            z.append(int(pax))
            llost.append(int(penality_compiled_old[fnames[k]]['lost_time'][i]/60))
            pen.append(int(penality_weight(wt,pax)))

        ax1.scatter(x,y,s=si)
        ax1.set_ylim([0,15])
        ax1.set_title('wait time in min')
        ax2.scatter(x,z,c='g',s=si)
        ax2.set_title('Pax Count')
        ax3.scatter(x,pen,c='r',s=si)
        ax3.set_title('Penality')
        ax4.scatter(x,llost,c='y',s=si)
        ax4.set_title('LLOST')

        x,y,z=[],[],[]
        pen,llost=[],[]
        for i in penality_compiled_new[fnames[k]]['wtime'].keys():
            wt=penality_compiled_new[fnames[k]]['wtime'][i]
            pax=penality_compiled_new[fnames[k]]['pax_cnt'][i]
            x.append(int(i/60))
            y.append(int(wt/60))
            z.append(int(pax))
            llost.append(int(penality_compiled_new[fnames[k]]['lost_time'][i]/60))
            pen.append(int(penality_weight(wt,pax)))

        #     ax5.set_xlim([0,18])
            ax5.scatter(x,y,s=si)
            ax5.set_ylim([0,15])
            ax5.set_title('wait time in min')

            ax6.scatter(x,z,c='g',s=si)
            ax6.set_title('Pax Count')
            ax7.scatter(x,pen,c='r',s=si)
            ax7.set_title('Penality')
            ax8.scatter(x,llost,c='y',s=si)
            ax8.set_title('LLOST')

            f.savefig("opti_data/PICS/"+fnames[k]+".png")
    except:
#         print("here")
        noy.append(k)